# Gas Transfer Coefficients

In [1]:
import numpy as np
import xarray as xr
import pandas as pd

In [2]:
# Molar Mass of Carbon in g/mmol
mC = 12.0108 * 1000 # (g . mol-1 )/(mmol . mol-1) 

In [3]:
def vapor_pressure_weiss1980(sss,sst_k):
    
    from numpy import exp, log
    
    T = sst_k
    S = sss
    
    # Equation comes straight from Weiss and Price (1980)
    pH2O = exp(+24.4543 - 67.4509 * (100 / T) - 4.8489 * log(T / 100) - 0.000544 * S)
    return pH2O

In [4]:
def vapor_pressure_dickson2007(sss,sst_k):
    
    from numpy import exp

    T = sst_k
    S = sss
    
    ###################################################
    # WATER VAPOUR PRESSURE FOR PURE WATER
    ###################################################
    # alpha coefficients from Wafner and Pruss, (2002)
    a1 = -7.85951783
    a2 = +1.84408259
    a3 = -11.7866497
    a4 = +22.6807411
    a5 = -15.9618719
    a6 = +1.80122502
    # critical points for water
    Pc = 22.064 / 101325.0e-6  # convert to atmosphers
    Tc = 647.096
    # zeta numbers correspond with alpha numbers
    z = 1 - T / Tc
    z1 = z
    z2 = z ** 1.5
    z3 = z ** 3
    z4 = z ** 3.5
    z5 = z ** 4
    z6 = z ** 7.5
    # vapour pressure of pure water
    pure_water = Pc * exp(
        (Tc / T) * (a1 * z1 + a2 * z2 + a3 * z3 + a4 * z4 + a5 * z5 + a6 * z6)
    )

    ###################################################
    # WATER VAPOUR PRESSURE FOR SEA WATER
    ###################################################
    # osmotic coeffcients at 25C - Millero 1974
    c0 = +0.90799
    c1 = -0.08992
    c2 = +0.18458
    c3 = -0.07395
    c4 = -0.00221
    # total molality of dissolved species
    total_molality = 31.998 * S / (1e3 - 1.005 * S)
    B1 = total_molality * 0.5
    B2 = B1 ** 2
    B3 = B1 ** 3
    B4 = B1 ** 4
    osmotic_coeff = c0 + c1 * B1 + c2 * B2 + c3 * B3 + c4 * B4

    seawater = pure_water * exp(-0.018 * osmotic_coeff * total_molality)

    return seawater
    

In [5]:
def calc_k0_weiss1974(sss,sst_k,press_atm=1):
    # SSS, SST in Kelvins, pressure in atmospheres
    from numpy import exp, log
    
    T = sst_k
    S = sss
    P = press_atm
    
    # from table in Wanninkhof 2014
    a1 = -58.0931
    a2 = +90.5069
    a3 = +22.2940
    b1 = +0.027766
    b2 = -0.025888
    b3 = +0.0050578
    
    T100 = T / 100
    K0 = exp(a1 + a2 * (100 / T) + a3 * log(T100) + S * (b1 + b2 * T100 + b3 * T100 ** 2))
    pH2O = vapor_pressure_dickson2007(S, T)
        
    K0 = K0 / (P - pH2O)
    return K0    # units of mol/L/atm

In [9]:
def calc_kw(wind_speed, wind_grid_stdev, temp_C, ice_frac, scaling=16):
    # wind in m/s, SST in Celsius
    
    from numpy import array, nansum, nan_to_num, isnan, around, nanmean
    
    Uavg = array(wind_speed)
    Ustd = array(wind_grid_stdev)
    T = array(temp_C)
    ice = array(ice_frac)

    Uavg2 = Uavg ** 2
    U2 = Uavg2 + (Ustd ** 2)
    
    #Sc = schmidt_number(T)
    aSc = +2116.8
    b = -136.25
    c = +4.7353
    d = -0.092307
    e = +0.0007555
    Sc = aSc + b * T + c * T ** 2 + d * T ** 3 + e * T ** 4
    
    Sc660 = (Sc / 660) ** -0.5
    
    mask = isnan(U2) | isnan(Sc)

    weight = nan_to_num(1 - ice)
    weight[mask] = 0

    a = around(scaling / (nansum(U2 * Sc660 * weight) / nansum(weight)), 4)
    #a = 0.276 #for ERA (2020)
    a = 0.271 # 2021
    
    kw = a * U2 * Sc660
         
    return kw   #cm/hr
    

In [7]:
def calc_flux(temp_bulk_C,salt_bulk,pCO2_bulk_uatm,pCO2_air_uatm,press_hPa,wind_ms,wind_grid_stdev,ice_frac):
            
    # piston velocity, solubility, delta pCO2, ice fraction
    from numpy import array, nanmean

    press_atm = array(press_hPa) / 1013.25

    SSTfnd_C = array(temp_bulk_C)
    SSTfnd_K = SSTfnd_C + 273.15

    SSSfnd = array(salt_bulk)

    pCO2sea = array(pCO2_bulk_uatm) * 1e-6  # to atm
    pCO2air = array(pCO2_air_uatm) * 1e-6
    
    K0blk = calc_k0_weiss1974(SSSfnd, SSTfnd_K, press_atm)
    
    # molar mas of carbon in g . mmol-1
    mC = 12.0108 * 1000  # (g . mol-1) / (mmol . mol-1)

    # KW : UNIT ANALYSIS
    # kw = (cm . hr-1) * hr . day-1 . cm-1 . m
    # kw = m . day-1
    kw = calc_kw(wind_ms, wind_grid_stdev, temp_bulk_C, ice_frac) 
    
    kw = kw * (24 / 100)
    
    # flux = (m . day-1) .  (mol . L-1 . atm-1) . atm . (gC . mmol-1)
    # flux = (m . day-1) . (mmol . m-3 . atm-1) . atm . (gC . mmol-1)
    # flux = gC . m-2 . day-1
    co2_flux = kw * K0blk * (pCO2sea - pCO2air) * mC 
            
    return co2_flux 